In [1]:
from pymongo import MongoClient
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px

In [3]:
mongo = MongoClient(port=27017)

In [4]:
print(mongo.list_database_names())

['Spotify_2023', 'admin', 'autosaurus', 'classDB', 'config', 'epa', 'local', 'met', 'petsitly_marketing', 'travel_db']


In [5]:
db = mongo['Spotify_2023']


In [6]:
print(db.list_collection_names())

['Song_list']


In [7]:
Most_Popular_Songs_2023 = db['Song_list']

In [8]:
Most_Popular_Songs_2023.find_one()

{'_id': ObjectId('666b907452d1c34f36f0c553'),
 'track_name': 'Seven (feat. Latto) (Explicit Ver.)',
 'artist(s)_name': 'Latto, Jung Kook',
 'artist_count': 2,
 'released_year': 2023,
 'released_month': 7,
 'released_day': 14,
 'in_spotify_playlists': 553,
 'in_spotify_charts': 147,
 'streams': 141381703,
 'in_apple_playlists': 43,
 'in_apple_charts': 263,
 'in_deezer_playlists': 45,
 'in_deezer_charts': 10,
 'in_shazam_charts': 826,
 'bpm': 125,
 'key': 'B',
 'mode': 'Major',
 'danceability_%': 80,
 'valence_%': 89,
 'energy_%': 83,
 'acousticness_%': 31,
 'instrumentalness_%': 0,
 'liveness_%': 8,
 'speechiness_%': 4}

In [9]:
x = list(Most_Popular_Songs_2023.find().sort({'in_spotify_charts':-1}))


In [10]:
df_Spotify = pd.DataFrame(x)
df_Spotify.head()

,_id,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,666b907452d1c34f36f0c553,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,...,125,B,Major,80,89,83,31,0,8,4
1,666b907452d1c34f36f0c561,As It Was,Harry Styles,1,2022,3,31,23575,130,2513188493,...,174,F#,Minor,52,66,73,34,0,31,6
2,666b907452d1c34f36f0c55f,Flowers,Miley Cyrus,1,2023,1,12,12211,115,1316855716,...,118,NaN,Major,71,65,68,6,0,3,7
3,666b907452d1c34f36f0c555,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,...,138,F,Major,51,32,53,17,0,31,6
4,666b907452d1c34f36f0c569,I Wanna Be Yours,Arctic Monkeys,1,2013,1,1,12859,110,1297026226,...,135,NaN,Minor,48,44,42,12,2,11,3


In [11]:
#to fix later, drop _id from dataframe
df_Spotify1 = df_Spotify.drop(columns=["danceability_%","valence_%","energy_%","acousticness_%","instrumentalness_%","liveness_%","speechiness_%","_id"])
df_Spotify1.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,263,45,10,826,125,B,Major
1,As It Was,Harry Styles,1,2022,3,31,23575,130,2513188493,403,198,863,46,NaN,174,F#,Minor
2,Flowers,Miley Cyrus,1,2023,1,12,12211,115,1316855716,300,215,745,58,"1,021",118,NaN,Major
3,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,207,91,14,949,138,F,Major
4,I Wanna Be Yours,Arctic Monkeys,1,2013,1,1,12859,110,1297026226,24,98,582,2,73,135,NaN,Minor


In [24]:
df_Spotify.info()

<class 'pandas.core.frame.DataFrame'>
Index: 895 entries, 0 to 952
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   _id                   0 non-null      float64
 1   track_name            895 non-null    object 
 2   artist(s)_name        895 non-null    object 
 3   artist_count          895 non-null    int64  
 4   released_year         895 non-null    int64  
 5   released_month        895 non-null    int64  
 6   released_day          895 non-null    int64  
 7   in_spotify_playlists  895 non-null    int64  
 8   in_spotify_charts     895 non-null    int64  
 9   streams               895 non-null    float64
 10  in_apple_playlists    895 non-null    int64  
 11  in_apple_charts       895 non-null    int64  
 12  in_deezer_playlists   829 non-null    float64
 13  in_deezer_charts      895 non-null    int64  
 14  in_shazam_charts      895 non-null    float64
 15  bpm                   895 no

In [12]:
df_Spotify1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    object
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 non-null    ob

In [26]:
df_Spotify = df_Spotify.head(100)
df_Spotify

,_id,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,NaN,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,1.413817e+08,...,125,3,0,80,89,83,31,0,8,4
3,NaN,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,1.400040e+08,...,138,8,0,51,32,53,17,0,31,6
4,NaN,I Wanna Be Yours,Arctic Monkeys,1,2013,1,1,12859,110,1.297026e+09,...,135,0,1,48,44,42,12,2,11,3
6,NaN,Dance The Night (From Barbie The Album),Dua Lipa,1,2023,5,25,2988,101,1.274090e+08,...,110,3,1,67,78,85,2,0,33,5
7,NaN,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,8.008408e+08,...,170,1,0,55,58,72,11,0,11,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,NaN,cÔøΩÔøΩmo dormi,Rels B,1,2022,8,4,1452,35,3.315114e+08,...,126,4,0,63,56,43,24,0,12,23
109,NaN,TULUM,"Peso Pluma, Grupo Frontera",2,2023,6,28,266,34,5.229427e+07,...,168,9,0,56,63,87,39,0,11,5
110,NaN,FrÔøΩÔøΩgil (feat. Grupo Front,"Yahritza Y Su Esencia, Grupo Frontera",2,2023,4,7,672,34,1.889335e+08,...,150,9,0,61,39,73,37,0,11,3
111,NaN,VAGABUNDO,"Sebastian Yatra, Manuel Turizo, BeÔøΩÔ",3,2023,5,12,1094,34,9.083975e+07,...,127,3,1,82,89,85,4,0,23,6


In [28]:
df_Spotify.info()


<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 113
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   _id                   0 non-null      float64
 1   track_name            100 non-null    object 
 2   artist(s)_name        100 non-null    object 
 3   artist_count          100 non-null    int64  
 4   released_year         100 non-null    int64  
 5   released_month        100 non-null    int64  
 6   released_day          100 non-null    int64  
 7   in_spotify_playlists  100 non-null    int64  
 8   in_spotify_charts     100 non-null    int64  
 9   streams               100 non-null    float64
 10  in_apple_playlists    100 non-null    int64  
 11  in_apple_charts       100 non-null    int64  
 12  in_deezer_playlists   89 non-null     float64
 13  in_deezer_charts      100 non-null    int64  
 14  in_shazam_charts      100 non-null    float64
 15  bpm                   100 no

In [29]:
df_Spotify.dropna(subset=['in_apple_charts', 'in_shazam_charts', 'in_spotify_charts', 'streams'], inplace=True)

df_Spotify['key'] = pd.Categorical(df_Spotify['key']).codes
df_Spotify['mode'] = pd.Categorical(df_Spotify['mode']).codes

fig_3d_scatter = px.scatter_3d(df_Spotify, x='in_apple_charts', y='in_shazam_charts', z='in_spotify_charts', color='streams',
                               size='streams', hover_name='track_name',
                               title='Chart Presence: Apple, Spotify, Shazam')
fig_3d_scatter.show()

/var/folders/5z/td30sfmn67xdb7gyg6n8vw8r0000gn/T/ipykernel_94646/14136453.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5z/td30sfmn67xdb7gyg6n8vw8r0000gn/T/ipykernel_94646/14136453.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5z/td30sfmn67xdb7gyg6n8vw8r0000gn/T/ipykernel_94646/14136453.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [13]:
categorical_columns = ['track_name', 'artist(s)_name', 'key', 'mode']
for col in df_Spotify.columns:
    if col not in categorical_columns:
        df_Spotify[col] = pd.to_numeric(df_Spotify[col], errors='coerce')

In [14]:
df_Spotify.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   _id                   0 non-null      float64
 1   track_name            953 non-null    object 
 2   artist(s)_name        953 non-null    object 
 3   artist_count          953 non-null    int64  
 4   released_year         953 non-null    int64  
 5   released_month        953 non-null    int64  
 6   released_day          953 non-null    int64  
 7   in_spotify_playlists  953 non-null    int64  
 8   in_spotify_charts     953 non-null    int64  
 9   streams               952 non-null    float64
 10  in_apple_playlists    953 non-null    int64  
 11  in_apple_charts       953 non-null    int64  
 12  in_deezer_playlists   874 non-null    float64
 13  in_deezer_charts      953 non-null    int64  
 14  in_shazam_charts      896 non-null    float64
 15  bpm                   9

In [18]:
df_Spotify.dropna(subset=['energy_%', 'valence_%', 'danceability_%', 'streams'], inplace=True)

df_Spotify['key'] = pd.Categorical(df_Spotify['key']).codes
df_Spotify['mode'] = pd.Categorical(df_Spotify['mode']).codes

fig_3d_scatter = px.scatter_3d(df_Spotify, x='energy_%', y='valence_%', z='danceability_%', color='streams',
                               size='streams', hover_name='track_name',
                               title='Chart Presence vs. Valence vs. Danceability')
fig_3d_scatter.show()